In [1]:
import findspark
findspark.init()
import pyspark 
from pyspark.sql import SparkSession

In [14]:
from pyspark.sql import SQLContext
from pyspark.sql import DataFrameNaFunctions
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer

In [3]:
spark = SparkSession.builder \
   .master("local") \
   .appName("daily_weather") \
   .config("spark.executor.memory", "6gb") \
   .getOrCreate()

In [5]:
df =spark.read.format('com.databricks.spark.csv')\
                    .options(header='true', inferschema='true')\
                    .load('daily_weather.csv')


In [6]:
df.columns

['number',
 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [7]:
featureColumns = [ 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [8]:
df = df.drop('number')

In [9]:
df = df.na.drop()

In [10]:
df.count(), len(df.columns)

(1064, 10)

In [16]:
binarizer = Binarizer(threshold = 24.999, inputCol = 'relative_humidity_3pm', outputCol = 'label')
binarizerDF = binarizer.transform(df)

In [18]:
binarizerDF.select('relative_humidity_3pm','label').show(4)

+---------------------+-----+
|relative_humidity_3pm|label|
+---------------------+-----+
|   36.160000000000494|  1.0|
|     19.4265967985621|  0.0|
|   14.460000000000045|  0.0|
|   12.742547353761848|  0.0|
+---------------------+-----+
only showing top 4 rows



In [21]:
assembler = VectorAssembler(inputCols = featureColumns, outputCol = 'features')
assembled = assembler.transform(binarizerDF)

In [22]:
(trainingData, testData) = assembled.randomSplit([0.8,0.2], seed = 13234)

In [23]:
trainingData.count(), testData.count()

(859, 205)

In [28]:
dt = DecisionTreeClassifier(labelCol = 'label', featuresCol = 'features', maxDepth = 5, minInstancesPerNode = 20, impurity = 'gini')

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "C:\spark\spark\python\pyspark\ml\wrapper.py", line 105, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'DecisionTreeClassifier' object has no attribute '_java_obj'
Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "C:\spark\spark\python\pyspark\ml\wrapper.py", line 105, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'DecisionTreeClassifier' object has no attribute '_java_obj'
Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "C:\spark\spark\python\pyspark\ml\wrapper.py", line 105, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'DecisionTreeClassifier' object has no attribute '_java_obj'


In [29]:
pipeline = Pipeline(stages = [dt])
model = pipeline.fit(trainingData)

In [ ]:
predictions = model.transform(testData)
predictions.select('prediction', 'label').show(10)